# Creating a Simple Agent with Tracing

In [9]:
import dotenv
import os

from openai import OpenAI
from notebooks.guardrails import nutrition_agent

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

ModuleNotFoundError: No module named 'notebooks.guardrails'

In [10]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [11]:
nutrition_agent = Agent(name="Nutrition Assistant", instructions="Give me nutrition facts for a given food.")

Let's execute the Agent:

In [12]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are a nutritious option. Here’s a typical profile:
    
    Per 100 g (about one small banana):
    - Calories: 89
    - Protein: 1.1 g
    - Fat: 0.3 g
    - Carbohydrates: 22.8 g
      - Sugars: 12.2 g
      - Fiber: 2.6 g
    - Potassium: 358 mg
    - Vitamin C: 8.7 mg
    - Vitamin B6: 0.37 mg
    
    Per medium banana (about 118 g):
    - Calories: ~105
    - Carbs: ~27 g
    - Fiber: ~3.1 g
    - Potassium: ~422 mg
    - Vitamin C: ~9 mg
    - Vitamin B6: ~0.4–0.5 mg
    
    How healthy it is:
    - Pros: good source of potassium, vitamin B6, vitamin C, and fiber; naturally low in fat; convenient and inexpensive.
    - Considerations: contains natural sugars; portion size matters if you’re watching calories or blood sugar. Suitable for most diets; people with bananas allergies or rare fructose intolerance should avoid.
    
    If you want, I can tailor this to a specific serving s

Streaming the answer to the screen, token by token

In [13]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are a nutritious, convenient fruit. For a medium banana (about 118 g):

- Calories: ~105
- Carbs: ~27 g
  - Sugars: ~14 g
  - Fiber: ~3 g
- Protein: ~1.3 g
- Fat: ~0.3 g
- Potassium: ~422 mg
- Vitamin B6: ~0.5 mg
- Vitamin C: ~10 mg

Health notes:
- Good source of potassium, vitamin B6, and fiber; supports heart and gut health.
- Moderate sugar content; glycemic index is in the medium range.
- Generally, part of a balanced diet; portion control matters for those managing blood sugar or total calorie intake.

Bottom line: bananas are a healthy, nutrient-dense fruit when eaten as part of a varied diet.

_Good Job!_